# Setup telegram

In [1]:
from telegram_bot_token import HTTP_TOKEN
import json
from pprint import PrettyPrinter
import telegram
pp = PrettyPrinter(indent=2)

In [2]:
bot = telegram.Bot(HTTP_TOKEN)
telegram_updates = bot.get_updates()
telegram_update = telegram_updates[0]

In [3]:
def notify_via_telegram():
    telegram_update.message.reply_text("Too much vibration is detected. Please watch have a look!")

In [4]:
notify_via_telegram()

# Setup BLE

In [5]:
import sys
from gattlib import GATTRequester
st_addres= "D5:C9:57:C2:25:7B"

requester = GATTRequester(st_addres, False)
requester.connect(True, channel_type="random")

In [6]:
requester.discover_primary()

[{'uuid': '00001801-0000-1000-8000-00805f9b34fb', 'start': 1, 'end': 4},
 {'uuid': '00001800-0000-1000-8000-00805f9b34fb', 'start': 5, 'end': 11},
 {'uuid': '00000000-0003-11e1-9ab4-0002a5d5c51b', 'start': 12, 'end': 17}]

In [7]:
characteristics = requester.discover_characteristics()
characteristics

[{'uuid': '00002a05-0000-1000-8000-00805f9b34fb',
  'handle': 2,
  'properties': 32,
  'value_handle': 3},
 {'uuid': '00002a00-0000-1000-8000-00805f9b34fb',
  'handle': 6,
  'properties': 78,
  'value_handle': 7},
 {'uuid': '00002a01-0000-1000-8000-00805f9b34fb',
  'handle': 8,
  'properties': 78,
  'value_handle': 9},
 {'uuid': '00002a04-0000-1000-8000-00805f9b34fb',
  'handle': 10,
  'properties': 2,
  'value_handle': 11},
 {'uuid': '20000000-0001-11e1-ac36-0002a5d5c51b',
  'handle': 13,
  'properties': 6,
  'value_handle': 14},
 {'uuid': '21000000-0001-11e1-ac36-0002a5d5c51b',
  'handle': 15,
  'properties': 18,
  'value_handle': 16}]

In [ ]:
hex(characteristics[-1]['properties'])

In [8]:
characteristics[-1]['properties'] & 0x10 == 0x10

True

![alt text](Properties.png "Title")

In [9]:
requester.disconnect()

In [10]:
class Requester(GATTRequester):
    def on_notification(self, handle, data):
        if handle == (self.notification_hanlde - 1):
            print('"Too Much Vibration" event is recieved from Nucleo via BLE.')
            notify_via_telegram()
    
    def register_for_notification(self):
        self.notification_hanlde = 0
        for char in characteristics:
            if char['uuid'] == '21000000-0001-11e1-ac36-0002a5d5c51b' and char['properties'] & 0x10 == 0x10:
                self.notification_hanlde = char['value_handle']+1
        self.write_by_handle(self.notification_hanlde, b"\x01\x00")
    
    def unregister_for_notification(self):
        self.write_by_handle(self.notification_hanlde, b"\x00\x00")

In [11]:
requester = Requester(st_addres, False)
requester.connect(True, channel_type="random")

In [12]:
requester.register_for_notification()

"Too Much Vibration" event is recieved from Nucleo via BLE.
"Too Much Vibration" event is recieved from Nucleo via BLE.
"Too Much Vibration" event is recieved from Nucleo via BLE.
"Too Much Vibration" event is recieved from Nucleo via BLE.
"Too Much Vibration" event is recieved from Nucleo via BLE.


In [13]:
requester.notification_hanlde

17

In [14]:
requester.unregister_for_notification()
requester.disconnect()